# perfSONAR testpoint installation and initial configuration
<p>the following steps will install the `perfsonar-testpoint` bundle</p>
<p>depending on the goals for this node, additional tasks include:
<ul>
<li>configuring this perfSONAR (pS) node to participate in a mesh</li>
<li>setting up this node to participate in GridFTP disk-to-disk testing</p>
</ul>

## pull in the perfsonar-testpoint notebook
<p>using your terminal app log into your fionette as root, and cd into /root</p>
<p>use wget to transfer the perfsonar-testpoint notebook<pre>
wget http://fiona-10-01:8000/perfsonar-testpoint.ipynb -O perfsonar-testpoint.ipynb
</pre>
<p>launch jupyter lab</p>
<pre>jupyter lab --allow-root --ip=`your.fionette.ip`</pre>

## first we will prepare the system for installing the `perfsonar-testpoint` bundle

In [ ]:
uname -r

In [ ]:
sed -i 's/SELINUX=enforcing/SELINUX=disabled/g' /etc/selinux/config
setenforce 0

## setup NTP, bind-utils, traceroute (if not already present)

In [ ]:
yum -y install ntp bind-utils traceroute

In [ ]:
systemctl enable ntpd

# adjust `/etc/ntp.conf` for the workshop NTP server
<p>review the contents of the file</p>

In [ ]:
cat /etc/ntp.conf

<p>using your favorite editor, comment out NTP servers outside of the workshop and add an entry for the local NTP server</p>
<pre>
#server time.apple.com
server 137.164.48.196 iburst
</pre>

## verify the changes

In [ ]:
cat /etc/ntp.conf

In [ ]:
systemctl start ntpd

In [ ]:
systemctl status ntpd -l

In [ ]:
ntpq -p -c rv

## determine default interface

In [ ]:
default_iface=$(awk '$2 == 00000000 { print $1 }' /proc/net/route)
echo $default_iface

## confirm `hostname -f` responds with FQHN

In [ ]:
hostname -f

## use hostnamectl to set if its not a valid FQHN

In [ ]:
myip=$(ip addr show dev $default_iface | grep 'inet '| awk '{print $2}' | awk -F "/" '{print $1}')
dig -x $myip +short

In [ ]:
hostnamectl set-hostname `dig -x $myip +short`
hostname -f

## view current link settings on default interface

In [ ]:
ip link show dev $default_iface
tc qdisc show

## create a custom udev rules file for tuning the NIC configuration 

In [ ]:
if [ ! -f /etc/udev/rules.d/80-prp-fiona-ws--ps-netconf.rules ]; then
cat > /etc/udev/rules.d/80-prp-fiona-ws--ps-netconf.rules <<EOL
KERNEL=="$default_iface", RUN+="/sbin/ip link set %k mtu 9000"
KERNEL=="$default_iface", RUN+="/sbin/ip link set %k txqueuelen 10000"
EOL
chmod 644 /etc/udev/rules.d/80-prp-fiona-ws--ps-netconf.rules
fi

## Use udevadm to reload the rules and trigger the action

In [ ]:
/sbin/udevadm control --reload-rules
/sbin/udevadm trigger

## confirm new NIC tuning settings are present

In [ ]:
ip link show dev $default_iface
tc qdisc show

## Create a custom sysctl file with desired settings

In [ ]:
if [ ! -f /etc/sysctl.d/80-prp-fiona-ws--ps-sysctl.conf ]; then
cat>/etc/sysctl.d/80-prp-fiona-ws--ps-sysctl.conf <<EOL
net.core.rmem_max = 67108864 
net.core.wmem_max = 67108864 
net.ipv4.tcp_rmem = 4096 87380 33554432
net.ipv4.tcp_wmem = 4096 65536 33554432
net.ipv4.tcp_congestion_control=htcp
EOL
fi
chmod 644 /etc/sysctl.d/80-prp-fiona-ws--ps-sysctl.conf

## Adjust kernel boot-time defaults and rebuild grub.cfg

In [ ]:
if ! grep --quiet default_hugepagesz /etc/default/grub; then
	sed -i 's/^GRUB_CMDLINE_LINUX=\"[^\"]*/& default_hugepagesz=1GB hugepagesz=1GB hugepages=8 elevator=noop processor.max_cstate=1 intel_iommu=off idle=poll intel_idle.max_cstate=0 nosoftlockup mce=ignore_mce/' /etc/default/grub
	grub2-mkconfig -o /boot/grub2/grub.cfg
fi

In [ ]:
cat /etc/default/grub

# SAVE your work NOW

In [ ]:
reboot

## confirm intended tuning settings are present 

In [ ]:
ip link show | grep UP | grep -v lo
tc qdisc show

## stop & disable a few un-wanted services and start & enable the `tuned` service

In [ ]:
systemctl stop NetworkManager 
systemctl disable NetworkManager
systemctl stop irqbalance.service
systemctl disable irqbalance.service
systemctl stop wpa_supplicant
systemctl disable wpa_supplicant
systemctl start tuned
systemctl enable tuned


## check the current clock rate of the CPU cores

In [ ]:
cat /proc/cpuinfo | grep Hz

## use `tuned-adm` to show the current active profile

In [ ]:
tuned-adm active

## set `network-throughput` at the active profile

In [ ]:
tuned-adm profile network-throughput

## re-check the clock rate of the CPU cores

In [ ]:
cat /proc/cpuinfo | grep Hz

## Install the perfSONAR testpoint bundle (perfsonar-testpoint) and additional tools

<p>The assumption is that CentOS 7 is installed on your machine and that you have access to the necessary tools and configuration to install the perfsonar-testpoint software bundle.</p>
<p>First we will install some addditional and supplementary tools, if not already present on the system</p>

## Installing the perfSONAR testpoint bundle
<p>The instructions below are based on the perfSONAR CentOS Bundle Installation page, here:</p>
<p>http://docs.perfsonar.net/install_centos.html</p>
<p>with modifications for this workshop</p>
## First we need to install the Extra Packages for Enterprise Linux (EPEL)

## point yum to local perfSONAR repo 

In [ ]:
cat << EOF > /etc/yum.repos.d/perfsonar.repo
[perfsonar]
name=Extra Packages for Enterprise Linux 7 - $basearch
baseurl=http://137.164.48.196/centos-7-x86_64/perfsonar
failovermethod=priority
enabled=1
gpgcheck=0
EOF

## Refresh the yum cache

In [ ]:
yum clean all

## Now we can proceed to install the `perfsonar-testpoint` bundle:
<p>NOTE: due to the extensive output from installing this bundle, the call to yum includes the `-q` switch to set quiet mode.  output to `stderr` and any non-zero exit code will appear in the cell below.</p>
<p>NOTE: in a terminal session, run <pre>tail -f /var/log/yum.log</pre> to track the bundle installation</p>

In [ ]:
yum -y -q install perfsonar-testpoint

## install optional `perfsonar-testpoint` packages

In [ ]:
yum -y install perfsonar-toolkit-ntp perfsonar-toolkit-security perfsonar-toolkit-sysctl perfsonar-toolkit-servicewatcher perfsonar-toolkit-systemenv-testpoint

## reload the firewall ruleset

In [ ]:
firewall-cmd --reload

## check current active firewall rules

In [ ]:
firewall-cmd --list-all

## NOTE: if the output of entry for `services:` entry does include `bwctl` or `owamp-control` re-run the script to configure the firewall rules for perfSONAR

In [ ]:
/usr/lib/perfsonar/scripts/configure_firewall install

## re-check firewall rules

In [ ]:
firewall-cmd --list-all

## use `systemctl` to check the status of processes

In [ ]:
systemctl status postgresql-9.5 -l
systemctl status httpd -l
systemctl status pscheduler* -l
systemctl status perfsonar-meshconfig-agent -l

## Save your work NOW
## Reboot the server and verify everything came up as expected
## You will need to re-login to your machine and launch the notebook

In [ ]:
reboot

In [ ]:
systemctl status postgresql-9.5 -l
systemctl status httpd -l
systemctl status pscheduler* -l
systemctl status perfsonar-meshconfig-agent -l

## update the `meshconfig-agent` to add a reference for the group mesh json
<p>review the current contents</p>

In [ ]:
cat /etc/perfsonar/meshconfig-agent.conf

## update the `meshconfig-agent.conf` with a mesh stanza referencing the group mesh json

In [ ]:
cd /etc/perfsonar
wget http://fiona-10-01:8000/host-to-group -O /usr/local/etc/prp-fiona-ws--host-to-group
mygrp=$(grep $(hostname -s) /usr/local/etc/prp-fiona-ws--host-to-group | awk '{print $4}')
if [ ! -f /etc/perfsonar/meshconfig-agent.conf.orig ]; then
cp /etc/perfsonar/meshconfig-agent.conf /etc/perfsonar/meshconfig-agent.conf.orig
fi
cat >> /etc/perfsonar/meshconfig-agent.conf <<EOL
<mesh>
   configuration_url http://gigiot-ma-$mygrp/prp-fiona-ws-mesh--gigiot-ma-$mygrp.json
   #validate_certificate         0
   #ca_certificate_file          /etc/pki/tls/certs/ca-bundle.crt
   configure_archives 1
</mesh>
EOL
chown perfsonar:perfsonar /etc/perfsonar/meshconfig-agent.conf
chmod 644 /etc/perfsonar/meshconfig-agent.conf

In [ ]:
cat /etc/perfsonar/meshconfig-agent.conf

## restart the `perfsonar-meshconfig-agent` process

In [ ]:
systemctl restart perfsonar-meshconfig-agent

## verify the contents of `meshconfig-agent-tasks.conf` to determine if the mesh json has been processed correctly
<p>it should have `<test>` stanzas which includes sections for test schedule, parameters, targets, and measurement archives</p> 

In [ ]:
cat /etc/perfsonar/meshconfig-agent-tasks.conf

# confirming a succesful installation and mesh participation


## open a terminal session and run the following to help determine the functionality of the system
<ul>
<li> `pscheduler monitor` to show an updating display of completed, running, and pending tasks</li>
<li> `pscheduler task throughput --dest some.other.ps.node` to run a throughput (iperf3) test</li>
<li> `pscheduler task latency --dest some.other.ps.node` to run a latency/loss (owping) test</li> 
</ul>

# perfSONAR testpoint appendix: perfSONAR testpoint installation troubleshooting 

## pscheduler processes fail to start 
<p>there are reports of a recent dependency breakage in the perfSONAR repo which may cause `httpd` to be absent from the bundle installation. pScheduler will not function without httpd.  if you attempt to use pscheduler and you receive an error message that says:<pre>Unable to find pScheduler on localhost</pre> this can be an indication that httpd is missing.</p>
<p>install and enable httpd if not present on the system</p>

In [ ]:
yum -y -q install httpd
systemctl enable httpd
systemctl start httpd
systemctl status httpd -l

## python library incompatibility on systems having both perSONAR and GridFTP
<p>We have seen an issue with the GridFTP installation where an incompatible python library is installed and this causes the pscheduler processes to not start. If your pscheduler-* processes do not start you can try the following:</p>

In [ ]:
pip uninstall urllib3
yum install -y python-urllib3
systemctl start pscheduler-*
systemctl status pscheduler-* -l

# Setting up scheduled GridFTP testing
<p>the following steps will set up cron jobs to accomplish</p>
<ul>
<li>a schedule of GridFTP transfers from a group of participating nodes</li>
<li>parsing the GridFTP transfer log and uploading the results to a central Measurement Archive (MA)</li>
</ul>

## wget the host-to-group map and the group-specific script for GridFTP and make it executable

In [ ]:
wget http://fiona-10-01:8000/host-to-group -O /usr/local/etc/prp-fiona-ws--host-to-group
mygrp=$(grep $(hostname -s) /usr/local/etc/prp-fiona-ws--host-to-group | awk '{print $4}')
wget http://fiona-10-01:8000/cron-load-gridftp-grp-0$mygrp.sh -O /usr/local/bin/cron-load-gridftp-grp-0$mygrp.sh
chmod 755 /usr/local/bin/cron-load-gridftp-grp-0$mygrp.sh

## inspect the contents of the script

In [ ]:
cat /usr/local/bin/cron-load-gridftp-grp-0$mygrp.sh

## using your favorite editor, update the `GRIDFTP_API_KEY` entry
<p>update the <pre>GRIDFTP_API_KEY</pre> entry with the API key generated earlier for the gridftp user on the MA for your group<p>

## Grab a helper script that gragefully times out the globus url copy if it exceeds a value in seconds.

In [ ]:
mkdir /opt/esmond-gridftp/
wget http://fiona-10-01:8000/timeout.sh -O /usr/local/bin/timeout.sh
chmod 755 /usr/local/bin/timeout.sh

## This script scrapes the gridftp transfer log for transfers that it has not yet processed
<p>The results are uploaded to the central measurement archive running on the MA host for your group.</p>

In [ ]:
/usr/local/bin/cron-load-gridftp-grp-0$mygrp.sh 


## Running this file the first time pulls in the cron-gridftp-transfer-mesh for this host

In [ ]:
chmod 644 /etc/cron.d/cron-gridftp-transfer-mesh-grp-0$mygrp

## Inspect the cron job

In [ ]:
cat /etc/cron.d/cron-gridftp-transfer-mesh-grp-0$mygrp